In [9]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json

# Paquetes de consulta a traves de Socrata

from sodapy import Socrata
identificador_columnas=pd.read_csv('datos/identificador_columnas.csv')
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
identificador_columnas=identificador_columnas[['Unificado','API_DA_S_I', 'API_DA_S_II']]
identificador_columnas=identificador_columnas.dropna()
identificador_columnas=identificador_columnas.set_index('Unificado')
identificador_columnas.drop('Fuente',axis=0,inplace=True)
identificador_datos={'SECOP I':'f789-7hwg','SECOP II':'jbjy-vk9h','TVEC':'rgxm-mmea'}
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
### Definición de cliente para Socrata

cliente=Socrata('www.datos.gov.co',None,timeout=1000000)

In [12]:
with open('datos/dptos.json') as json_file:
    dptos = json.load(json_file)

In [13]:
def consulta_departamento(departamento):
    results_detalle=[]
    for i in ['SECOP I','SECOP II']:
        queries_list=[]
        results_detalle_dpto=[]
        for j in dptos[departamento]:
            query_secop_tot=""" Select
                {0} as id_contrato,
                {1} as id_proceso,
                {2} as valor_contrato,
                {3} as entidad,
                {4} as departamento_entidad,
                {5} as orden,
                {6} as modalidad,
                {7} as tipo_contrato,
                {8} as codigo_unspsc,
                {9} as objeto_contrato,
                {10} as descripcion_proceso,
                {11} as nombre_proveedor,
                {12} as tipo_documento_proveedor,
                {13} as numero_documento_proveedor,
                {14} as fecha_inicio,
                {15} as fecha_fin,
                {16} as fecha_firma,
                {17} as url

            where
                {4}='{18}'
            limit
                10000000
            """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['ID Proceso'],
            identificador_columnas[valor_columnas[i]]['Valor del contrato'],
            identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
            identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['Modalidad'],
            identificador_columnas[valor_columnas[i]]['Tipo de contrato'],identificador_columnas[valor_columnas[i]]['UNSPSC'],
            identificador_columnas[valor_columnas[i]]['Objeto Contrato'],identificador_columnas[valor_columnas[i]]['Descripción proceso'],
            identificador_columnas[valor_columnas[i]]['Nombre Proveedor'],identificador_columnas[valor_columnas[i]]['Tipo de documento proveedor'],
            identificador_columnas[valor_columnas[i]]['Documento Proveedor'],identificador_columnas[valor_columnas[i]]['Inicio de contrato'],
            identificador_columnas[valor_columnas[i]]['Fin de contrato'],identificador_columnas[valor_columnas[i]]['Fecha de Firma'],
            identificador_columnas[valor_columnas[i]]['URL'],j)
            queries_list.append(query_secop_tot)
            df_temp=pd.DataFrame(cliente.get(identificador_datos[i], query=query_secop_tot))
            df_temp['fuente']=i
            if df_temp.shape[0]>0:
                results_detalle_dpto.append(df_temp)
        if len(results_detalle_dpto)>0:
            results_detalle.append(pd.concat(results_detalle_dpto))
    results_detalle=pd.concat(results_detalle)
    results_detalle.reset_index(inplace=True)
    results_detalle.drop('index',axis=1,inplace=True)
    results_detalle['valor_contrato']=results_detalle['valor_contrato'].astype(float)
    results_detalle['departamento_entidad']=results_detalle['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
      
    return results_detalle

In [22]:
def consulta_datos_departamento(departamento,año):
    df_consulta=pd.read_csv('datos/contratos_entidad.csv')
    df_consulta=df_consulta[df_consulta['departamento_entidad']==departamento]
    df_consulta=df_consulta[df_consulta['año']==año]
    df_consulta['suma_valor_contrato']=df_consulta['suma_valor_contrato'].astype(float)
    df_consulta['cantidad']=df_consulta['cantidad'].astype(int)
    df_consulta['departamento_entidad']=df_consulta['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
    return df_consulta

In [24]:
consulta_datos_departamento('Bogotá DC',2021)

,entidad,nit_entidad,año,orden,departamento_entidad,cantidad,suma_valor_contrato
15,ADMINISTRADORA COLOMBIANA DE PENSIONES COLPENS...,900336004,2021,NACIONAL,Bogotá DC,14,9.899384e+10
24,ADMINISTRADORA DE LOS RECURSOS DEL SISTEMA GEN...,901037916,2021,NACIONAL,Bogotá DC,731,2.546268e+10
46,AERONÁUTICA CIVIL AEROCIVIL,899999059,2021,NACIONAL,Bogotá DC,14,1.305194e+10
51,AGENCIA DE COMERCIALIZACION E INOVACION PARA E...,901486723,2021,TERRITORIAL,Bogotá DC,57,2.197213e+10
53,AGENCIA DE CUNDINAMARCA PARA LA PAZ Y LA CONVI...,901024331,2021,TERRITORIAL,Bogotá DC,3,1.773173e+08
...,...,...,...,...,...,...,...
78448,UNIDAD PARA LAS VÍCTIMAS FONDO PARA LA REPARA...,900490473,2021,NACIONAL,Bogotá DC,119,3.836412e+10
78463,UNIDAD PRESTADORA DE SALUD BOGOTÁ,901361596,2021,NACIONAL,Bogotá DC,263,6.187687e+09
78515,UNIVERSIDAD MILITAR NUEVA GRANADA,800225340,2021,NACIONAL,Bogotá DC,498,2.459611e+10
78524,UP HOLDING SAS,900828603,2021,CORPORACIÓN AUTÓNOMA,Bogotá DC,16,1.711615e+09
